<a href="https://colab.research.google.com/github/Rogerio-mack/fNIRS/blob/main/A_Carol_create_mean_tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & import `nilearn` and `mne-nirs`

In [20]:
#@markdown
!pip install nilearn
!pip install -U --no-deps https://github.com/mne-tools/mne-python/archive/main.zip
!pip install mne-nirs

from mne.io import read_raw_nirx
from mne.preprocessing.nirs import (optical_density, beer_lambert_law,
                                    temporal_derivative_distribution_repair)
import mne 



  Using cached https://github.com/mne-tools/mne-python/archive/main.zip


## Mount Google Drive

In [21]:
#@markdown
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Common Imports

In [22]:
#@markdown 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns

import h5py
from itertools import compress
from sklearn.preprocessing import scale
from google.colab import widgets

# Copia todos arquivos do drive para local

In [23]:
import os
path_in = '/content/drive/MyDrive/Cond 2/snirf/snirf0/'
# path_in = '/content/drive/MyDrive/cond 1/snirf/snirf0/'

if 'cond 1' in path_in:
  condicao = 1
else:
  condicao = 2

path_out = '/content/'

In [24]:
import pickle

# To retrieve pickle
filename = path_in + 'raw_haemo_all.pickle'
infile = open(filename,'rb')
raw_haemo_all = pickle.load(infile)
infile.close()

filename = path_in + 'epochs_all.pickle'
infile = open(filename,'rb')
epochs_all = pickle.load(infile)
infile.close()



In [25]:
df_raw_haemo_all = {}

for k in raw_haemo_all:
  df_raw_haemo_all[k] = raw_haemo_all[k].to_data_frame()

df_epochs_all = {}

for k in epochs_all:
  df_epochs_all[k] = epochs_all[k].to_data_frame()


In [26]:
df_raw_haemo_all[list( df_raw_haemo_all.keys() )[0]].head()

,time,S1_D1 hbo,S1_D1 hbr,S1_D8 hbo,S1_D8 hbr,S1_D14 hbo,S1_D14 hbr,S2_D2 hbo,S2_D2 hbr,S2_D3 hbo,...,S5_D14 hbo,S5_D14 hbr,S5_D15 hbo,S5_D15 hbr,S6_D10 hbo,S6_D10 hbr,S6_D12 hbo,S6_D12 hbr,S6_D13 hbo,S6_D13 hbr
0,0,2.981556e-13,8.131516e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.312901e-14,1.694066e-15,3.642242e-14,6.352747e-16,0.0,0.0,5.886879e-14,8.470329e-15,3.769297e-14,-2.752857e-15
1,100,-2.171236e+03,-9.524455e+02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-8.023409e-01,1.588825e+01,2.130439e+00,5.042658e+00,0.0,0.0,-4.705156e+00,1.540719e+01,-4.062256e+00,4.250733e+01
2,200,-4.154342e+03,-1.822240e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.920337e+00,3.004193e+01,2.743334e+00,9.257077e+00,0.0,0.0,-1.035819e+01,2.905519e+01,-8.782348e+00,8.003903e+01
3,300,-5.789499e+03,-2.539129e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.503622e+00,4.095947e+01,5.146604e-01,1.191335e+01,0.0,0.0,-1.779615e+01,3.942277e+01,-1.480082e+01,1.084348e+02
4,400,-6.968332e+03,-3.055491e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.539116e+01,4.756638e+01,-5.521605e+00,1.246796e+01,0.0,0.0,-2.764718e+01,4.542406e+01,-2.270134e+01,1.249912e+02


In [27]:
df_epochs_all[list( df_epochs_all.keys() )[0]].head()

,time,condition,epoch,S1_D1 hbo,S1_D1 hbr,S1_D8 hbo,S1_D8 hbr,S1_D14 hbo,S1_D14 hbr,S2_D2 hbo,...,S5_D14 hbo,S5_D14 hbr,S5_D15 hbo,S5_D15 hbr,S6_D10 hbo,S6_D10 hbr,S6_D12 hbo,S6_D12 hbr,S6_D13 hbo,S6_D13 hbr
0,-5000,Sync1,0,-39.190545,-17.486756,0.0,0.0,0.0,0.0,0.0,...,-7.065572,-3.667420,-6.127343,-3.905441,0.0,0.0,-5.827807,-1.741747,-9.061614,-2.813122
1,-4900,Sync1,0,-33.007835,-16.305755,0.0,0.0,0.0,0.0,0.0,...,-6.610221,-3.616116,-5.748079,-3.700896,0.0,0.0,-5.594543,-1.560074,-8.599687,-2.721034
2,-4800,Sync1,0,-27.217239,-15.155453,0.0,0.0,0.0,0.0,0.0,...,-6.171506,-3.481363,-5.398483,-3.466396,0.0,0.0,-5.314016,-1.401514,-8.164702,-2.573892
3,-4700,Sync1,0,-21.889123,-14.002135,0.0,0.0,0.0,0.0,0.0,...,-5.743135,-3.264127,-5.067838,-3.212423,0.0,0.0,-4.979131,-1.277129,-7.728344,-2.415512
4,-4600,Sync1,0,-17.067956,-12.834718,0.0,0.0,0.0,0.0,0.0,...,-5.322585,-2.977779,-4.743853,-2.954412,0.0,0.0,-4.595017,-1.192104,-7.271991,-2.279432


In [28]:
df_raw_haemo_mean = df_raw_haemo_all[list( df_raw_haemo_all.keys() )[0]]

for c in df_raw_haemo_mean.drop(columns=['time']):
  df_raw_haemo_mean[c] = 0

df_epochs_mean = df_epochs_all[list( df_epochs_all.keys() )[0]]

for c in df_epochs_mean.drop(columns=['time','condition','epoch']):
  df_epochs_mean[c] = 0

In [29]:
for k in df_raw_haemo_all:
  for c in df_raw_haemo_all[k].drop(columns=['time']):
    df_raw_haemo_mean[c] = df_raw_haemo_mean[c] + df_raw_haemo_all[k][c]

for c in df_raw_haemo_mean.drop(columns=['time']):
    df_raw_haemo_mean[c] = df_raw_haemo_mean[c] / len( df_raw_haemo_all.keys() )
  

In [30]:
for k in df_epochs_all:
  for c in df_epochs_all[k].drop(columns=['time','condition','epoch']):
    df_epochs_mean[c] = df_epochs_mean[c] + df_epochs_all[k][c]

for c in df_epochs_mean.drop(columns=['time','condition','epoch']):
    df_epochs_mean[c] = df_epochs_mean[c] / len( df_epochs_all.keys() )

In [31]:
df_raw_haemo_mean.head()

,time,S1_D1 hbo,S1_D1 hbr,S1_D8 hbo,S1_D8 hbr,S1_D14 hbo,S1_D14 hbr,S2_D2 hbo,S2_D2 hbr,S2_D3 hbo,...,S5_D14 hbo,S5_D14 hbr,S5_D15 hbo,S5_D15 hbr,S6_D10 hbo,S6_D10 hbr,S6_D12 hbo,S6_D12 hbr,S6_D13 hbo,S6_D13 hbr
0,0,4.536164e-14,-2.083550e-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.658085e-14,-3.164273e-14,1.105529e-13,3.738289e-14,0.0,0.0,-1.698471e-13,3.057638e-14,-5.118499e-14,8.432516e-14
1,100,-3.137539e+02,-7.738659e+02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.329283e+03,-4.441251e+02,-1.272831e+03,-5.252833e+02,0.0,0.0,-1.232430e+03,-5.059191e+02,-1.216955e+03,-4.987121e+02
2,200,-6.004985e+02,-1.480640e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.543849e+03,-8.496104e+02,-2.435600e+03,-1.005157e+03,0.0,0.0,-2.359078e+03,-9.683422e+02,-2.328772e+03,-9.544121e+02
3,300,-8.372789e+02,-2.063333e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.546237e+03,-1.183650e+03,-3.394811e+03,-1.401044e+03,0.0,0.0,-3.290019e+03,-1.350302e+03,-3.246151e+03,-1.330535e+03
4,400,-1.008539e+03,-2.483319e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.270370e+03,-1.424043e+03,-4.087104e+03,-1.686799e+03,0.0,0.0,-3.964192e+03,-1.626682e+03,-3.908576e+03,-1.602284e+03


In [32]:
df_epochs_mean.head()

,time,condition,epoch,S1_D1 hbo,S1_D1 hbr,S1_D8 hbo,S1_D8 hbr,S1_D14 hbo,S1_D14 hbr,S2_D2 hbo,...,S5_D14 hbo,S5_D14 hbr,S5_D15 hbo,S5_D15 hbr,S6_D10 hbo,S6_D10 hbr,S6_D12 hbo,S6_D12 hbr,S6_D13 hbo,S6_D13 hbr
0,-5000,Sync1,0,-4.940127,-16.890077,0.0,0.0,0.0,0.0,0.0,...,-23.303281,-7.741267,-23.170225,-9.868259,0.0,0.0,-24.814744,-12.073923,-29.818632,-4.021063
1,-4900,Sync1,0,-2.844324,-15.426689,0.0,0.0,0.0,0.0,0.0,...,-20.500394,-6.314867,-20.388463,-8.525104,0.0,0.0,-21.146907,-11.755421,-28.552431,-1.042988
2,-4800,Sync1,0,-1.355425,-13.840101,0.0,0.0,0.0,0.0,0.0,...,-17.770962,-5.125166,-17.766887,-7.239282,0.0,0.0,-17.715769,-11.243363,-27.160018,1.509442
3,-4700,Sync1,0,-0.411415,-12.101137,0.0,0.0,0.0,0.0,0.0,...,-15.032789,-4.266698,-15.293080,-6.021165,0.0,0.0,-14.557279,-10.525158,-25.456877,3.457221
4,-4600,Sync1,0,0.103686,-10.229380,0.0,0.0,0.0,0.0,0.0,...,-12.247226,-3.768485,-12.956714,-4.872947,0.0,0.0,-11.683642,-9.635345,-23.314713,4.694303


In [33]:
df_raw_haemo_mean.to_excel(path_in + 'haemo_mean' + '.xlsx',index=None)
df_epochs_mean.to_excel(path_in + 'epochs_mean' + '.xlsx',index=None)